In [4]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-t3qqh0xp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-t3qqh0xp
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=7b155e36f975e615cd6ec9e48124d2f63dc881cef3b39c2cce72422a65bd4627
  Stored in directory: /tmp/pip-ephem-wheel-cache-wp6ayh5e/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [22]:
! pip install open_clip_torch


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/results.py", line 184, in __init__
    self[name] = toklist[0]
TypeError: 'int' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrappe

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import open_clip
import os
import clip
import torch
from PIL import Image

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
#model, preprocess = clip.load('ViT-B/32', device)

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-16')
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-16')

#Data loading

image_folder = '/content/drive/MyDrive/Unique_BACH_folder'  # Specifica il percorso alla cartella delle immagini
image_files = [f for f in os.listdir(image_folder) if f.endswith(('tif'))]

import pandas as pd

# Leggi il file Excel
df = pd.read_csv('/content/drive/MyDrive/WE/microscopy_ground_truth.csv')

# Inizializza la lista per le coppie
y_vere = []

import numpy as np

y_pred = []

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


templates = ["a histopathology slide showing {}",
"histopathology image of {}",
"pathology tissue showing {}",
"presence of {} tissue on image"]

classnames = ["Breast normal breast tissue", "Breast non-malignant benign tissue", "Breast malignant in-situ carcinoma", "Breast malignant invasive carcinoma"]

clip_weights = []
for classname in classnames:
    texts = []
    # Tokenize the prompts
    classname = classname.replace('_', ' ')
    for t in templates:
      texts.append(t.format(classname))

    text_tokens = clip.tokenize(texts).to(device)
    class_embeddings = model.encode_text(text_tokens)
    class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
    class_embeddings /= class_embeddings.norm(dim=1, keepdim=True)
    clip_weights.append(class_embeddings)
print(len(clip_weights))
clip_weights = torch.stack(clip_weights, dim=-1).to(device)
print(clip_weights.shape)
text_features = clip_weights.mean(dim=0)
print(text_features.shape)

texts = ["Breast normal breast tissue", "Breast non-malignant benign tissue", "Breast malignant in-situ carcinoma", "Breast malignant invasive carcinoma"]
"""
with torch.no_grad():
  text = tokenizer(texts).to(device)
  text_features = model.encode_text(text)
  print(text_features.shape)
  text_features /= text_features.norm(dim=-1, keepdim=True)
  print("THE SHAAAAAAAAAAAAAAAAAAPE---------")
  print(text_features.shape)
"""
y_code = {
    "Normal": 0,
    "Benign": 1,
    "InSitu": 2,
    "Invasive": 3
}

all_pred = []
all_y = []

for index, row in df.iterrows():
    coppia = (row[0], row[1])  # Leggi la colonna 0 e la colonna 1 e crea la coppia
    image_file = row[0]
    yt = row[1]
    y_int = y_code[yt]

    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)

    # Preprocessa l'immagine usando le trasformazioni di validazione
    image_preprocessed = preprocess_val(image).unsqueeze(0)  # aggiungi una dimensione batch
    image_preprocessed = image_preprocessed.to(device)

    with torch.no_grad():
      image_features = model.encode_image(image_preprocessed)
      image_features /= image_features.norm(dim=-1, keepdim=True)

      text_probs = (100.0 * image_features @ text_features).softmax(dim=-1)
      y_hat = text_probs.argmax(dim=-1).item()

    all_pred.append(y_hat)
    all_y.append(y_int)

    acc = ((np.array(all_pred) - np.array(all_y)) == 0).mean() * 100.
    print(f"Accuracy: {acc:.2f}%")



4
torch.Size([4, 512, 4])
torch.Size([512, 4])
Accuracy: 100.00%
Accuracy: 50.00%
Accuracy: 33.33%
Accuracy: 25.00%
Accuracy: 40.00%
Accuracy: 50.00%
Accuracy: 57.14%
Accuracy: 62.50%
Accuracy: 66.67%
Accuracy: 70.00%
Accuracy: 72.73%
Accuracy: 66.67%
Accuracy: 69.23%
Accuracy: 71.43%
Accuracy: 73.33%
Accuracy: 75.00%
Accuracy: 70.59%
Accuracy: 66.67%
Accuracy: 68.42%
Accuracy: 70.00%
Accuracy: 71.43%
Accuracy: 72.73%
Accuracy: 73.91%
Accuracy: 75.00%
Accuracy: 72.00%
Accuracy: 73.08%
Accuracy: 70.37%
Accuracy: 67.86%
Accuracy: 68.97%
Accuracy: 66.67%
Accuracy: 67.74%
Accuracy: 68.75%
Accuracy: 69.70%
Accuracy: 67.65%
Accuracy: 65.71%
Accuracy: 66.67%
Accuracy: 67.57%
Accuracy: 65.79%
Accuracy: 66.67%
Accuracy: 65.00%
Accuracy: 63.41%
Accuracy: 61.90%
Accuracy: 62.79%
Accuracy: 61.36%
Accuracy: 62.22%
Accuracy: 60.87%
Accuracy: 59.57%
Accuracy: 60.42%
Accuracy: 61.22%
Accuracy: 60.00%
Accuracy: 58.82%
Accuracy: 57.69%
Accuracy: 58.49%
Accuracy: 59.26%
Accuracy: 60.00%
Accuracy: 58.93%


In [15]:

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)



In [ ]:
#New code
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

In [4]:
#/QuiltNet-B-32 ALREADY EXECUTED--------------------------------------------------------------
from PIL import Image
import requests
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32') #16
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

In [5]:
import torch
from PIL import Image

In [6]:
import os
image_folder = '/content/drive/MyDrive/Unique_BACH_folder'  # Specifica il percorso alla cartella delle immagini
image_files = [f for f in os.listdir(image_folder) if f.endswith(('tif'))]

In [7]:
import pandas as pd

# Leggi il file Excel
df = pd.read_csv('/content/drive/MyDrive/WE/microscopy_ground_truth.csv')


# Inizializza la lista per le coppie
y_vere = []


#-----------------------------------------------------DOVREBBE ESSERCI IL PROBLEMA
# Itera sul DataFrame e crea le coppie
import numpy as np

y_pred = []

# Carica il modello e le trasformazioni
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32')

# Prepara il testo
#corispondenza classe t0, t1, t2, t3
texts = ["Breast normal breast tissue", "Breast non-malignant benign tissue", "Breast malignant in-situ carcinoma", "Breast malignant invasive carcinoma"]
#texts = ["Normal breast tissue histology shows well-organized lobules and ducts lined by a double layer of epithelial and myoepithelial cells. The stroma is composed of dense fibrous connective tissue interspersed with adipose tissue. The epithelial cells are uniform in appearance with no signs of atypia or mitotic activity. Myoepithelial cells are present at the periphery of the ducts and lobules, demonstrating the normal architecture. No inflammatory cells or pathological changes are observed. The tissue exhibits a normal distribution of glandular and stromal components, indicative of healthy breast tissue.","benign","in-situ", "invasive"]

tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')


for index, row in df.iterrows():
    coppia = (row[0], row[1])  # Leggi la colonna 0 e la colonna 1 e crea la coppia
    image_file = row[0]
    yt = row[1]
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)

    # Preprocessa l'immagine usando le trasformazioni di validazione
    image_preprocessed = preprocess_val(image).unsqueeze(0)  # aggiungi una dimensione batch

    # Tokenizza il testo e calcola la lunghezza massima dei token per ogni batch di testi
    tokenized_texts = [tokenizer.encode(text) for text in texts]
    max_len = max(len(tokens) for tokens in tokenized_texts)

    # Padding dinamico dei testi per ogni batch
    padded_texts = [torch.nn.functional.pad(torch.tensor(tokens), (0, max_len - len(tokens)), 'constant', 0) for tokens in tokenized_texts]
    text_inputs = torch.stack(padded_texts)

    # Sposta il modello e gli input su GPU se disponibile
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    image_preprocessed = image_preprocessed.to(device)
    text_inputs = text_inputs.to(device)

    # Passa gli input preprocessati al modello
    with torch.no_grad():
        image_features = model.encode_image(image_preprocessed)

        # Truncate or pad the text inputs to match the expected length of positional embeddings
        expected_length = model.positional_embedding.shape[0] # Get the expected length from the model
        text_inputs = torch.nn.functional.pad(text_inputs, (0, expected_length - text_inputs.shape[1]), 'constant', 0)  # Pad if shorter
        text_inputs = text_inputs[:, :expected_length]  # Truncate if longer

        text_features = model.encode_text(text_inputs)

    # Calcola le similarità tra immagine e testi
    logits_per_image = (image_features @ text_features.T).softmax(dim=-1) #----------------------

    max_value, max_index = torch.max(logits_per_image, dim=1)
    posizione = "" #valore di supporto per fare cose


    if (max_index.item() == 3):
      posizione = "Invasive"
    elif (max_index.item() == 2):
      posizione = "InSitu"
    elif (max_index.item() == 1):
      posizione = "Benign"
    else:
      posizione = "Normal"


    #Aggiungere gli elementi ad una lista con soli nomi + labels predette
    y_pred.append((yt, posizione))
    print(y_pred[-1])




('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'InSitu')
('Normal', 'Invasive')
('Normal', 'InSitu')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'InSitu')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invasive')
('Normal', 'Invas

In [10]:
label = []
labelp = []
for (y,yp) in y_pred:
  if y== "Invasive":
      label.append(0)
  elif y== "InSitu":
      label.append(1)
  elif y== "Benign":
      label.append(2)
  else:
      label.append(3)
  if yp== "Invasive":
      labelp.append(0)
  elif yp== "InSitu":
      labelp.append(1)
  elif yp== "Benign":
      labelp.append(2)
  else:
      labelp.append(3)

In [11]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label, labelp))


[[63 37  0  0]
 [92  8  0  0]
 [91  4  4  1]
 [86 12  1  1]]


In [12]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(label, labelp))


0.19


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label, labelp))


[[39 58  0  3]
 [49 46  3  2]
 [41 28  9 22]
 [39 36 17  8]]


In [ ]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(label, labelp))


0.255


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label, labelp))


[[20 76  0  4]
 [19 77  1  3]
 [13 77  5  5]
 [ 4 90  3  3]]


In [ ]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(label, labelp))


0.2625


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label, labelp))


[[ 0 97  3  0]
 [ 0 97  0  3]
 [ 0 90  3  7]
 [ 0 93  1  6]]


In [ ]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(label, labelp))


0.265


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label, labelp))


[[63 37  0  0]
 [92  8  0  0]
 [91  4  4  1]
 [86 12  1  1]]


In [ ]:
import numpy as np

y_pred = []

# Carica il modello e le trasformazioni
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32')

# Prepara il testo
#corispondenza classe t0, t1, t2, t3

texts = ["Breast normal breast tissue","Breast non-malignant benign tissue","Breast malignant in-situ carcinoma", "Breast malignant invasive carcinoma",]
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')

# Itera su ogni immagine nel folder
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)

    # Preprocessa l'immagine usando le trasformazioni di validazione
    image_preprocessed = preprocess_val(image).unsqueeze(0)  # aggiungi una dimensione batch

    # Tokenizza il testo e calcola la lunghezza massima dei token per ogni batch di testi
    tokenized_texts = [tokenizer.encode(text) for text in texts]
    max_len = max(len(tokens) for tokens in tokenized_texts)

    # Padding dinamico dei testi per ogni batch
    padded_texts = [torch.nn.functional.pad(torch.tensor(tokens), (0, max_len - len(tokens)), 'constant', 0) for tokens in tokenized_texts]
    text_inputs = torch.stack(padded_texts)

    # Sposta il modello e gli input su GPU se disponibile
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    image_preprocessed = image_preprocessed.to(device)
    text_inputs = text_inputs.to(device)

    # Passa gli input preprocessati al modello
    with torch.no_grad():
        image_features = model.encode_image(image_preprocessed)

        # Truncate or pad the text inputs to match the expected length of positional embeddings
        expected_length = model.positional_embedding.shape[0] # Get the expected length from the model
        text_inputs = torch.nn.functional.pad(text_inputs, (0, expected_length - text_inputs.shape[1]), 'constant', 0)  # Pad if shorter
        text_inputs = text_inputs[:, :expected_length]  # Truncate if longer

        text_features = model.encode_text(text_inputs)

    # Calcola le similarità tra immagine e testi
    logits_per_image = (image_features @ text_features.T).softmax(dim=-1) #----------------------

    max_value, max_index = torch.max(logits_per_image, dim=1)
    posizione = "" #valore di supporto per fare cose


    if (max_index.item() == 3):
      posizione = "Invasive"
    elif (max_index.item() == 2):
      posizione = "InSitu"
    elif (max_index.item() == 1):
      posizione = "Benign"
    else:
      posizione = "Normal"


    #Aggiungere gli elementi ad una lista con soli nomi + labels predette
    y_pred.append((image_file, posizione))






    #Stampa i risultati
    #print(f"Logits per immagine {image_file}:" )
    #print(f"Il valore massimo è {max_value.item()} e si trova in posizione {max_index.item()}")
    #print("\n")

KeyboardInterrupt: 

In [ ]:
#Ora stampo il mio vettore di labels prodotteeeeeeeeeeeeee
print("Valori PREDETTI ----------------------------------------------------------------")

for coppia in y_pred:
  if coppia[0] ==  "n001.tif":
    print("Ci sono")
  else:
    print(coppia)
    print("\n")

print("La lunghezza della lista PRED è:", len(y_pred)) #400 IL MISTERO SI INFITTISCE




Valori PREDETTI ----------------------------------------------------------------
('is013.tif', 'Invasive')


('is007.tif', 'Invasive')


('is004.tif', 'Invasive')


('is010.tif', 'Invasive')


('is011.tif', 'Invasive')


('is005.tif', 'Invasive')


('is039.tif', 'InSitu')


('is015.tif', 'Invasive')


('is014.tif', 'Invasive')


('is016.tif', 'Invasive')


('is002.tif', 'Invasive')


('is058.tif', 'Invasive')


('is064.tif', 'Invasive')


('is065.tif', 'Invasive')


('is071.tif', 'Invasive')


('is067.tif', 'Invasive')


('is099.tif', 'Invasive')


('is066.tif', 'Invasive')


('is062.tif', 'InSitu')


('is063.tif', 'Invasive')


('is075.tif', 'InSitu')


('is061.tif', 'InSitu')


('is049.tif', 'Invasive')


('is074.tif', 'Invasive')


('is079.tif', 'Invasive')


('is100.tif', 'InSitu')


('is051.tif', 'Invasive')


('is087.tif', 'Invasive')


('is050.tif', 'Invasive')


('is046.tif', 'Invasive')


('is052.tif', 'Invasive')


('is085.tif', 'Invasive')


('is091.tif', 'Invasive')


('is0

CODICE CHE LAVORA SULLE LABELS VERE:

In [ ]:
import pandas as pd

# Leggi il file Excel
df = pd.read_csv('/content/drive/MyDrive/WE/microscopy_ground_truth.csv')


# Inizializza la lista per le coppie
y_vere = []


#-----------------------------------------------------DOVREBBE ESSERCI IL PROBLEMA
# Itera sul DataFrame e crea le coppie

for index, row in df.iterrows():
    coppia = (row[0], row[1])  # Leggi la colonna 0 e la colonna 1 e crea la coppia
    y_vere.append(coppia)  # Aggiungi la coppia alla lista

# Stampa la lista di coppie


print("Valori reali ----------------------------------------------------------------")

for coppia in y_vere:
      print(coppia)
      print("\n")




print("La lunghezza della lista VERE è:", len(y_vere)) #400


#CODCIE FIXATOOOOOOOOOOOO

Valori reali ----------------------------------------------------------------
C'e anche il 001-------------------------------
('n002.tif', 'Normal')


('n003.tif', 'Normal')


('n004.tif', 'Normal')


('n005.tif', 'Normal')


('n006.tif', 'Normal')


('n007.tif', 'Normal')


('n008.tif', 'Normal')


('n009.tif', 'Normal')


('n010.tif', 'Normal')


('n011.tif', 'Normal')


('n012.tif', 'Normal')


('n013.tif', 'Normal')


('n014.tif', 'Normal')


('n015.tif', 'Normal')


('n016.tif', 'Normal')


('n017.tif', 'Normal')


('n018.tif', 'Normal')


('n019.tif', 'Normal')


('n020.tif', 'Normal')


('n021.tif', 'Normal')


('n022.tif', 'Normal')


('n023.tif', 'Normal')


('n024.tif', 'Normal')


('n025.tif', 'Normal')


('n026.tif', 'Normal')


('n027.tif', 'Normal')


('n028.tif', 'Normal')


('n029.tif', 'Normal')


('n030.tif', 'Normal')


('n031.tif', 'Normal')


('n032.tif', 'Normal')


('n033.tif', 'Normal')


('n034.tif', 'Normal')


('n035.tif', 'Normal')


('n036.tif', 'Normal')



Sezione del codice in cui, dopo aver verificato la corretta creazione delle due liste andiamo a calcolare l'accuracyyyyyyyy

In [ ]:
#NON COMPILARE PIUUUUUUUUUUUUU SOLO PER BACKUP DI CODICE

#Andiamo a ripulire il contenuto di y_vere
y_vere = y_vere[:-2]
#verifico la composizione della lista a cui ho inserito l'elemento. Consiglio di fare l'inserimento in una cella separata in maniera tale da aggiungerlo solo quando necessario<
for coppia in y_vere:
    print(coppia)
    print("\n")
#Da qui in poi non modificheremo piu il contenuto di y_vere

('n002.tif', 'Normal')


('n003.tif', 'Normal')


('n004.tif', 'Normal')


('n005.tif', 'Normal')


('n006.tif', 'Normal')


('n007.tif', 'Normal')


('n008.tif', 'Normal')


('n009.tif', 'Normal')


('n010.tif', 'Normal')


('n011.tif', 'Normal')


('n012.tif', 'Normal')


('n013.tif', 'Normal')


('n014.tif', 'Normal')


('n015.tif', 'Normal')


('n016.tif', 'Normal')


('n017.tif', 'Normal')


('n018.tif', 'Normal')


('n019.tif', 'Normal')


('n020.tif', 'Normal')


('n021.tif', 'Normal')


('n022.tif', 'Normal')


('n023.tif', 'Normal')


('n024.tif', 'Normal')


('n025.tif', 'Normal')


('n026.tif', 'Normal')


('n027.tif', 'Normal')


('n028.tif', 'Normal')


('n029.tif', 'Normal')


('n030.tif', 'Normal')


('n031.tif', 'Normal')


('n032.tif', 'Normal')


('n033.tif', 'Normal')


('n034.tif', 'Normal')


('n035.tif', 'Normal')


('n036.tif', 'Normal')


('n037.tif', 'Normal')


('n038.tif', 'Normal')


('n039.tif', 'Normal')


('n040.tif', 'Normal')


('n041.tif', 'Normal')




In [ ]:
#una volta ottenute le due liste, voglio andare ad ordinarmi la lista valori reali

#confornto prima di tutto se le due liste contengono lo stesso numero di elementi
if len(y_pred) == len(y_vere):
    print("Le due liste hanno la stessa lunghezza.")
else:
    print("Le due liste hanno lunghezze diverse.")


#andiamo a debuggare -- - vabbe------------------:)---mandiamo avanti il codice RICORDA DI FIXAREEE
print("La lunghezza della lista PRED è:", len(y_pred))
print("La lunghezza della lista VERE è:", len(y_vere))



# Inizializzazione della lista di coppie ordinata
ORDINATA = []


#STACOSANONFUNZIONA
my_counter = 0


Le due liste hanno la stessa lunghezza.
La lunghezza della lista PRED è: 400
La lunghezza della lista VERE è: 400


In [ ]:
# Converti le liste in set per poter fare l'operazione di differenza
set_b = set(y_vere)
set_a = set(y_pred)

# Trova le coppie che sono in B ma non in A
differenza = set_b - set_a

# Converti il risultato in una lista (se necessario)
differenza_lista = list(differenza)

# Stampa la differenza
print("Coppie presenti in B ma non in A:", differenza_lista)


Coppie presenti in B ma non in A: [('n061.tif', 'Normal'), ('is078.tif', 'InSitu'), ('iv068.tif', 'Invasive'), ('is022.tif', 'InSitu'), ('is024.tif', 'InSitu'), ('b025.tif', 'Benign'), ('iv077.tif', 'Invasive'), ('n007.tif', 'Normal'), ('b019.tif', 'Benign'), ('b024.tif', 'Benign'), ('n095.tif', 'Normal'), ('n013.tif', 'Normal'), ('iv098.tif', 'Invasive'), ('n088.tif', 'Normal'), ('is091.tif', 'InSitu'), ('is050.tif', 'InSitu'), ('n096.tif', 'Normal'), ('b083.tif', 'Benign'), ('n008.tif', 'Normal'), ('b068.tif', 'Benign'), ('b012.tif', 'Benign'), ('iv053.tif', 'Invasive'), ('n078.tif', 'Normal'), ('is093.tif', 'InSitu'), ('b007.tif', 'Benign'), ('b049.tif', 'Benign'), ('is042.tif', 'InSitu'), ('b053.tif', 'Benign'), ('n063.tif', 'Normal'), ('n032.tif', 'Normal'), ('b099.tif', 'Benign'), ('b046.tif', 'Benign'), ('n074.tif', 'Normal'), ('is053.tif', 'InSitu'), ('n098.tif', 'Normal'), ('n003.tif', 'Normal'), ('n028.tif', 'Normal'), ('n070.tif', 'Normal'), ('n085.tif', 'Normal'), ('iv002.t

In [ ]:
for coppia in y_vere: #Il ciclo viene operato sulla stringa piu corta ed e`per questo che le due stringhe vengono di lunghezza uguale pur non essendolo.!!!!!!!-------------
    ticerco = coppia[0]  # Estrai il primo elemento della coppia corrente di A e salvalo in ticerco
    #print("\n", my_counter) #segnala l'avvenuta aggiunta di un elemento alla lista ordinata
    # Secondo ciclo for per cercare ticerco in tutti i primi elementi delle coppie presenti nella lista di coppie B
    for coppia_b in y_pred:
        # Se trovi ticerco in uno dei primi elementi delle coppie di B
        if ticerco == coppia_b[0]:
            # Aggiungi la coppia di B trovata nella lista ORDINATA
            ORDINATA.append(coppia_b)
            my_counter = my_counter + 1

print("LA LISTA PREDETTA ORDINATA e'", )

for coppia in ORDINATA:
    print(coppia)
    print("\n")



LA LISTA PREDETTA ORDINATA e'
('n001.tif', 'Invasive')


('n002.tif', 'Invasive')


('n003.tif', 'Invasive')


('n004.tif', 'Invasive')


('n005.tif', 'InSitu')


('n006.tif', 'Invasive')


('n007.tif', 'InSitu')


('n008.tif', 'Invasive')


('n009.tif', 'Invasive')


('n010.tif', 'Invasive')


('n011.tif', 'Invasive')


('n012.tif', 'Invasive')


('n013.tif', 'Invasive')


('n014.tif', 'Invasive')


('n015.tif', 'Invasive')


('n016.tif', 'Invasive')


('n017.tif', 'Invasive')


('n018.tif', 'Invasive')


('n019.tif', 'Invasive')


('n020.tif', 'Invasive')


('n021.tif', 'Invasive')


('n022.tif', 'Invasive')


('n023.tif', 'Invasive')


('n024.tif', 'Invasive')


('n025.tif', 'Invasive')


('n026.tif', 'Invasive')


('n027.tif', 'Invasive')


('n028.tif', 'Invasive')


('n029.tif', 'Invasive')


('n030.tif', 'Invasive')


('n031.tif', 'Invasive')


('n032.tif', 'Invasive')


('n033.tif', 'Invasive')


('n034.tif', 'Invasive')


('n035.tif', 'Invasive')


('n036.tif', 'Invasive')


('

In [ ]:
#Quale è l'elemento che non ha un simile?

print("La lunghezza della lista PRED è:", len(ORDINATA)) #399
print("La lunghezza della lista VERE è:", len(y_vere)) #400
#stampa le lunghezze delle due stringhe per
i = 0
index = 0

# fai un for della lunghezza della lista piu lunga, in questo caso predicted labels

for coppia in ORDINATA:
    # Estrai il primo elemento dalla coppia corrente in l1
    elemento_l1 = coppia[0]
    i = index
    for pippo in y_vere:
        # Estrai il primo elemento dalla prima coppia in l2
        elemento_l2 = pippo[0] #perchè pippoi mi da solo lettere????????!
        #print(elemento_l2) #-------------------------------------------
        # Confronta i primi elementi
        if elemento_l1 == elemento_l2:
            print("Match")
            index = index+1
    if(i == index): #singifica che non è avvenuto l'aggiornamento
      print("Nessuna corrispondenza per: \n", elemento_l1)


La lunghezza della lista PRED è: 400
La lunghezza della lista VERE è: 400
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Ma

In [ ]:
#Per sicurezza stampati la lista A e B in un file di testo per compararle agevolmente
# Definizione della lista di coppie


# Nome del file di output
nome_file_p = '/content/drive/MyDrive/WE/lista_coppie_pred.txt'
nome_file_v = '/content/drive/MyDrive/WE/lista_coppie_vere.txt'

def scrivi_lista_in_file(lista_coppie, nome_file):
    try:
        with open(nome_file, 'w') as file:
            for coppia in lista_coppie:
                file.write(f"{coppia[0]}, {coppia[1]}\n")
        print(f"Lista di coppie scritta in {nome_file}")
    except Exception as e:
        print(f"Errore durante la scrittura nel file: {e}")

scrivi_lista_in_file(ORDINATA, nome_file_p)

scrivi_lista_in_file(y_vere, nome_file_v)



Lista di coppie scritta in /content/drive/MyDrive/WE/lista_coppie_pred.txt
Lista di coppie scritta in /content/drive/MyDrive/WE/lista_coppie_vere.txt


In [ ]:
#Ora posso estrarre le labels della prima e della seconda e calcolare l'accuracy

#estraggo le labels reali
reallabels = []
predictedlabels = []

# Iterazione sulla lista di coppie
for coppia in y_vere:
    # Aggiungi il secondo elemento della coppia alla lista reallabels
    reallabels.append(coppia[1])

# Stampare la lista reallabels
print(reallabels)

#estraggo le labels predette----------------------------------------------------------
# Iterazione sulla lista di coppie
for coppia in ORDINATA:
    # Aggiungi il secondo elemento della coppia alla lista reallabels
    predictedlabels.append(coppia[1])

# Stampare la lista reallabels
print(predictedlabels)


['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',

In [ ]:
len(reallabels), len(predictedlabels)

(400, 400)

In [ ]:
from sklearn.metrics import accuracy_score

#look at the parameters and their meaning
accuracy_score(reallabels, predictedlabels)

0.19